In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(42)

Load train, val and test dataset

In [4]:
target_col = 'counter'

In [7]:
import pandas as pd

df_all = pd.read_csv(
    'data/all_emails_categorical_hour.csv',
)
df_train = pd.read_csv(
    'data/train_emails_categorical_hour.csv',
)
df_val = pd.read_csv(
    'data/val_emails_categorical_hour.csv',
)
df_test = pd.read_csv(
    'data/tuesday_27_june_emails_categorical_hour.csv',
)

In [8]:
df_train

,counter,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,...,month_9,month_10,month_11,month_12,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,75,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,88,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16312,2,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
16313,4,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
16314,43,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
16315,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
from torch.utils.data import Dataset

class EnronDataset(Dataset):
    def __init__(self, x, y):
        if not torch.is_tensor(x):
            x = torch.from_numpy(x)
        if not torch.is_tensor(y):
            y = torch.from_numpy(y)
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, i):
        return self.x[i], self.y[i]

In [ ]:
train_cols = list(df_all.columns)

x_train = df_train[train_cols].to_numpy()
y_train = df_train[target_col].to_numpy()

x_val = df_val[train_cols].to_numpy()
y_val = df_val[target_col].to_numpy()

x_test = df_test[train_cols].to_numpy()
y_test = df_test[target_col].to_numpy()

In [ ]:
trainloader = torch.utils.data.DataLoader(
    EnronDataset(x_train, y_train), 
    batch_size=2**10, # TODO check this
    shuffle=True, 
    num_workers=1 # TODO check this
    )

Define a Regression Neural Network

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(195, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

Define a Loss function and optimizer

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=1e-4) 

start training

In [ ]:
epochs = 3000



In [ ]:
loss=0
current_loss=0
loss_list=[]
for epoch in range(0, epochs):
    
    for i, data in enumerate(trainloader,0):
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape(((targets.shape[0], 1)))
        
        optimizer.zero_grad()
        outputs = net(inputs)
        
        loss= loss_f(outputs, targets)
        
        loss.backward()
        current_loss += loss.item()
        optimizer.step()
        
    if(epoch%1==0 or epoch==epochs-1):
        l = current_loss/i
        print('epoch {}, loss {}'.format(epoch, l))
        loss_list.append(l)
        current_loss=0

Predict

In [ ]:
with torch.no_grad():
    predicted = net(x_test)

y_test = y_test.detach().numpy()
predicted = predicted.reshape(-1,1)
predicted = predicted.detach().numpy()
predicted = np.int_(predicted)
predicted = predicted.T[0]

i=0
for a in predicted:
    if a<0:
        predicted[i]=0
    i+=1

print("PREDETTI")
print(predicted)
print("VERI")
print(y_test)

Err=(ln.norm(y_test-predicted,ord=2)/ln.norm(y_test,2))*100
print("Errore")
print(Err)